In [1]:
import openai
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import random
import re
import time
import pickle
import copy
import json
import requests
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
filepath=r'./'
plotpath=r'./'
assert os.path.exists(filepath)

# analysis

In [2]:
df=pd.read_excel(os.path.join(filepath,'survey_result.xlsx'))
df['scale name'].fillna(method='ffill', inplace=True)
df.shape

(495, 17)

In [3]:
df.head()

,scale name,id,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,self reflection scale,1,3,4,2,3,2,3,3,1,2,1,3,6,2,1,2
1,self reflection scale,2,2,3,1,2,2,3,4,2,3,2,2,1,3,2,2
2,self reflection scale,3,5,5,5,5,4,5,5,4,4,5,5,6,5,5,5
3,self reflection scale,4,2,2,2,2,3,3,2,5,2,3,2,1,3,4,5
4,self reflection scale,5,5,6,6,5,5,5,4,6,5,6,5,6,6,5,6


In [215]:
df['scale name'].unique().tolist()

['self reflection scale',
 'Insight Scale',
 'Risk-Taking',
 'Risk-Perception',
 'Risk Propensity Scale',
 'Emotion Regulation Questionnaire (ERQ)',
 'Need for Cognition',
 'Rationality inventory (REI-R)',
 'Cognitive Reflection Test',
 'Defeasible Reasoning',
 'Wason Selection Task',
 'General Decision-Making Style',
 'Letter Sets Test',
 'Logiqa 2.0',
 'Causal Reasoning Test on LLM',
 'Scientific Reasoning Scale',
 'Critical Thinking Disposition Scale',
 'Availability Heuristics',
 'Base-Rate Neglect (Statistical)',
 'Base-Rate Neglect (Causal)',
 'Framing Effect: Risk Framing',
 'Framing Effect: Attribute Framing',
 'Outcome bias',
 'Conjunction Fallacy',
 'Probabilistic Matching',
 'Sunk Cost Fallacy',
 'Regret Aversion',
 'V2 Defeasible Reasoning',
 'V2 Wason Selection Task',
 'Actively Open-Minded Thinking Scale',
 'Belief Bias in Syllogistic Reasoning',
 'Bias Blind Spot',
 'Hindsight Bias',
 'Illusion of control',
 'Overconfidence',
 'Loss Aversion',
 'Temporal Discounting',
 '

## self reflection scale

In [25]:
scale_name='self reflection scale'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
ids=[1,2,4,7]
dfscale.loc[ids] = 7 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,4,3,5,4,5,4,4,6,5,6,4,1,5,6,5
2,5,4,6,5,5,4,3,5,4,5,5,6,4,5,5
3,5,5,5,5,4,5,5,4,4,5,5,6,5,5,5
4,5,5,5,5,4,4,5,2,5,4,5,6,4,3,2
5,5,6,6,5,5,5,4,6,5,6,5,6,6,5,6
6,5,5,5,5,4,5,5,1,6,6,4,6,6,6,5
7,5,5,6,5,4,4,4,5,6,6,5,6,5,4,6
8,6,6,6,5,5,4,5,3,5,5,5,6,6,6,6
9,5,5,5,5,6,5,5,4,6,5,5,6,6,6,6


In [29]:
# Engagement in Self-Reflection 
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[1,2,3,4,5,6]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val <= 3 else 'High' if val >= 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,29,28,32,29,27,27,26,24,29,32,28,31,30,30,28
1,4.83,4.67,5.33,4.83,4.5,4.5,4.33,4.0,4.83,5.33,4.67,5.17,5.0,5.0,4.67
2,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [31]:
# Need for Self-Reflection
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[7,8,9,10,11,12]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val <= 3 else 'High' if val >= 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,34,33,34,30,32,28,31,22,34,34,31,36,35,34,36
1,5.67,5.5,5.67,5.0,5.33,4.67,5.17,3.67,5.67,5.67,5.17,6.0,5.83,5.67,6.0
2,High,High,High,High,High,High,High,Neutral,High,High,High,High,High,High,High


In [32]:
# Self-reflection
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[1,2,3,4,5,6,7,8,9,10,11,12]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val <= 3 else 'High' if val >= 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,63,61,66,59,59,55,57,46,63,66,59,67,65,64,64
1,5.25,5.08,5.5,4.92,4.92,4.58,4.75,3.83,5.25,5.5,4.92,5.58,5.42,5.33,5.33
2,High,High,High,High,High,High,High,Neutral,High,High,High,High,High,High,High


## Insight Scale

In [33]:
scale_name='Insight Scale'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
ids=[2,4,5,6,7]
dfscale.loc[ids] = 7 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,5,6,5,5,5,5,5,5,5,5,5,5,6,6,2
2,5,4,4,5,4,2,4,3,5,1,5,4,5,4,5
3,5,5,6,5,4,4,4,6,4,4,4,5,6,5,5
4,4,3,5,3,3,2,5,2,4,2,4,3,3,3,2
5,5,5,6,5,5,2,6,1,5,1,5,4,1,4,2
6,5,4,5,6,5,1,5,3,6,1,6,4,1,3,2
7,4,3,4,5,4,0,4,2,4,1,3,3,1,3,1
8,5,6,5,5,5,5,4,6,6,4,5,5,6,6,6


In [35]:
# Insight
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[1,2,3,4,5,6,7,8]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val <= 3 else 'High' if val >= 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,38,36,40,39,35,21,37,28,39,19,37,33,29,34,25
1,4.75,4.5,5.0,4.88,4.38,2.62,4.62,3.5,4.88,2.38,4.62,4.12,3.62,4.25,3.12
2,High,High,High,High,High,Low,High,Neutral,High,Low,High,High,Neutral,High,Neutral


## Risk-Taking

In [42]:
scale_name='Risk-Taking'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id').astype(float)

# # reverse scoring
# ids=[2,4,5,6,7]
# dfscale.loc[ids] = 7 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
# dfscale

In [43]:
# Ethical
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[6,9,10,16,29,30]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,6.0,8.0,6.0,11.0,6.0,9.0,8.0,22.0,13.0,32.0,9.0,13.0,18.0,16.0,36.0
1,1.0,1.33,1.0,1.83,1.0,1.5,1.33,3.67,2.17,5.33,1.5,2.17,3.0,2.67,6.0
2,Low,Low,Low,Low,Low,Low,Low,Low,Low,High,Low,Low,Low,Low,High


In [44]:
# Financial
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[3,4,8,12,14,18]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,20.0,16.0,15.0,23.0,19.0,22.0,22.0,35.0,24.0,32.0,19.0,19.0,22.0,19.0,41.0
1,3.33,2.67,2.5,3.83,3.17,3.67,3.67,5.83,4.0,5.33,3.17,3.17,3.67,3.17,6.83
2,Low,Low,Low,Low,Low,Low,Low,High,Neutral,High,Low,Low,Low,Low,High


In [45]:
# Health/Safety
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[5,15,17,20,23,26]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,8.0,11.0,13.0,17.0,17.0,14.0,12.0,16.0,6.0,21.0,13.0,17.0,14.0,13.0,37.0
1,1.33,1.83,2.17,2.83,2.83,2.33,2.0,2.67,1.0,3.5,2.17,2.83,2.33,2.17,6.17
2,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,High


In [46]:
# Recreational
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[2,11,13,19,24,25]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,26.0,33.0,32.0,36.0,22.0,23.0,21.0,23.0,25.0,38.0,18.0,34.0,16.0,23.0,42.0
1,4.33,5.5,5.33,6.0,3.67,3.83,3.5,3.83,4.17,6.33,3.0,5.67,2.67,3.83,7.0
2,High,High,High,High,Low,Low,Low,Low,High,High,Low,High,Low,Low,High


In [47]:
# Social
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[1,7,21,22,27,28]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,33.0,40.0,35.0,29.0,35.0,30.0,35.0,30.0,34.0,34.0,34.0,25.0,31.0,33.0,37.0
1,5.5,6.67,5.83,4.83,5.83,5.0,5.83,5.0,5.67,5.67,5.67,4.17,5.17,5.5,6.17
2,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [48]:
# overall
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,93.0,108.0,101.0,116.0,99.0,98.0,98.0,126.0,102.0,157.0,93.0,108.0,101.0,104.0,193.0
1,3.1,3.6,3.37,3.87,3.3,3.27,3.27,4.2,3.4,5.23,3.1,3.6,3.37,3.47,6.43
2,Low,Low,Low,Low,Low,Low,Low,High,Low,High,Low,Low,Low,Low,High


In [ ]:
0	93.0	108.0	101.0	116.0	99.0	98.0	98.0	126.0	102.0	157.0	93.0	108.0	101.0	0.0	193.0
1	3.1	3.6	3.37	3.87	3.3	3.27	3.27	4.2	3.4	5.23	3.1	3.6	3.37	0.0	6.43
2	Low	Low	Low	Low	Low	Low	Low	High	Low	High	Low	Low	Low	Low	High

## Risk-Perception

In [49]:
scale_name='Risk-Perception'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id').astype(float)

# # reverse scoring
# ids=[2,4,5,6,7]
# dfscale.loc[ids] = 7 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
# dfscale

In [50]:
# Ethical
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[6,9,10,16,29,30]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,36.0,34.0,35.0,35.0,32.0,32.0,32.0,10.0,39.0,37.0,32.0,38.0,34.0,36.0,15.0
1,6.0,5.67,5.83,5.83,5.33,5.33,5.33,1.67,6.5,6.17,5.33,6.33,5.67,6.0,2.5
2,High,High,High,High,High,High,High,Low,High,High,High,High,High,High,Low


In [51]:
# Financial
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[3,4,8,12,14,18]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,27.0,33.0,33.0,29.0,30.0,28.0,28.0,22.0,32.0,35.0,34.0,34.0,33.0,32.0,28.0
1,4.5,5.5,5.5,4.83,5.0,4.67,4.67,3.67,5.33,5.83,5.67,5.67,5.5,5.33,4.67
2,High,High,High,High,High,High,High,Low,High,High,High,High,High,High,High


In [52]:
# Health/Safety
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[5,15,17,20,23,26]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,36.0,33.0,23.0,33.0,30.0,35.0,33.0,20.0,32.0,34.0,32.0,34.0,36.0,40.0,15.0
1,6.0,5.5,3.83,5.5,5.0,5.83,5.5,3.33,5.33,5.67,5.33,5.67,6.0,6.67,2.5
2,High,High,Low,High,High,High,High,Low,High,High,High,High,High,High,Low


In [53]:
# Recreational
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[2,11,13,19,24,25]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,29.0,32.0,24.0,35.0,32.0,30.0,34.0,18.0,30.0,38.0,30.0,34.0,36.0,34.0,15.0
1,4.83,5.33,4.0,5.83,5.33,5.0,5.67,3.0,5.0,6.33,5.0,5.67,6.0,5.67,2.5
2,High,High,Neutral,High,High,High,High,Low,High,High,High,High,High,High,Low


In [54]:
# Social
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[1,7,21,22,27,28]
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,18.0,19.0,21.0,23.0,15.0,24.0,20.0,10.0,14.0,23.0,17.0,31.0,25.0,23.0,15.0
1,3.0,3.17,3.5,3.83,2.5,4.0,3.33,1.67,2.33,3.83,2.83,5.17,4.17,3.83,2.5
2,Low,Low,Low,Low,Low,Neutral,Low,Low,Low,Low,Low,High,High,Low,Low


In [55]:
# overall
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc[0] = dfscale.loc[ids].sum()
dfresult.loc[1] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc[1]
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc[2] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
0,146.0,151.0,136.0,155.0,139.0,149.0,147.0,80.0,147.0,167.0,145.0,171.0,164.0,165.0,88.0
1,4.87,5.03,4.53,5.17,4.63,4.97,4.9,2.67,4.9,5.57,4.83,5.7,5.47,5.5,2.93
2,High,High,High,High,High,High,High,Low,High,High,High,High,High,High,Low


## Risk Propensity Scale

In [58]:
scale_name='Risk Propensity Scale'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id').astype(float)

# reverse scoring
ids=[4]
dfscale.loc[ids] = 8 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
# dfscale

In [59]:
# Risk Propensity Scale
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,25.0,16.0,29.0,11.0,30.0,27.0,26.0,22.0,21.0,13.0,21.0,8.0,35.0,19.0,23.0
Average_score,5.0,3.2,5.8,2.2,6.0,5.4,5.2,4.4,4.2,2.6,4.2,1.6,7.0,3.8,4.6
Level,High,Low,High,Low,High,High,High,High,High,Low,High,Low,High,Low,High


## Emotion Regulation Questionnaire (ERQ)

In [60]:
scale_name='Emotion Regulation Questionnaire (ERQ)'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id').astype(float)

# reverse scoring
# ids=[4]
# dfscale.loc[ids] = 8 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
# dfscale

In [61]:
# Reappraisal Items
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[1,3,5,7,8,10]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,36.0,36.0,42.0,38.0,34.0,32.0,37.0,41.0,41.0,37.0,41.0,36.0,36.0,40.0,30.0
Average_score,6.0,6.0,7.0,6.33,5.67,5.33,6.17,6.83,6.83,6.17,6.83,6.0,6.0,6.67,5.0
Level,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [62]:
# Suppression Items
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[2,4,6,9]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,12.0,10.0,24.0,20.0,10.0,20.0,13.0,10.0,6.0,15.0,7.0,19.0,25.0,22.0,21.0
Average_score,3.0,2.5,6.0,5.0,2.5,5.0,3.25,2.5,1.5,3.75,1.75,4.75,6.25,5.5,5.25
Level,Low,Low,High,High,Low,High,Low,Low,Low,Low,Low,High,High,High,High


In [63]:
# Overall
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 4 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,48.0,46.0,66.0,58.0,44.0,52.0,50.0,51.0,47.0,52.0,48.0,55.0,61.0,62.0,51.0
Average_score,4.8,4.6,6.6,5.8,4.4,5.2,5.0,5.1,4.7,5.2,4.8,5.5,6.1,6.2,5.1
Level,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [ ]:
Total_score	48.0	46.0	66.0	58.0	44.0	52.0	50.0	51.0	47.0	52.0	48.0	55.0	61.0	62.0	51.0
Average_score	4.8	4.6	6.6	5.8	4.4	5.2	5.0	5.1	4.7	5.2	4.8	5.5	6.1	6.2	5.1
Level	High	High	High	High	High	High	High	High	High	High	High	High	High	High	High

## Need for Cognition

In [51]:
scale_name='Need for Cognition'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
ids=[3,4,5,7,8,9,12,16,17]
dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale.shape

(18, 15)

In [52]:
# Overall
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,71,78,84,68,71,83,74,77,77,72,71,72,69,67,63
Average_score,3.94,4.33,4.67,3.78,3.94,4.61,4.11,4.28,4.28,4.0,3.94,4.0,3.83,3.72,3.5
Level,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [ ]:
Total_score	71	78	84	68	71	83	74	77	77	72	71	72	69	67	63
Average_score	3.94	4.33	4.67	3.78	3.94	4.61	4.11	4.28	4.28	4.0	3.94	4.0	3.83	3.72	3.5
Level	High	High	High	High	High	High	High	High	High	High	High	High	High	High	High

## Rationality inventory (REI-R)

In [64]:
scale_name='Rationality inventory (REI-R)'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
ids=[1,2,4,5,7,8,9,11,12,18,22,29,30,32,33,34,36,37,40]

dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale.shape

(40, 15)

In [65]:
# Rational Engagement
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[1,3,5,6,7,10,11,15,18,20]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,44,41,44,39,40,46,37,41,48,43,39,36,36,37,42
Average_score,4.4,4.1,4.4,3.9,4.0,4.6,3.7,4.1,4.8,4.3,3.9,3.6,3.6,3.7,4.2
Level,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [66]:
# Rational Ability: 
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[2,4,8,9,12,13,14,16,17,19]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,40,36,41,37,34,44,34,40,45,36,41,40,36,36,45
Average_score,4.0,3.6,4.1,3.7,3.4,4.4,3.4,4.0,4.5,3.6,4.1,4.0,3.6,3.6,4.5
Level,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [67]:
# Rational Overall
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=list(range(1,21))
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,84,77,85,76,74,90,71,81,93,79,80,76,72,73,87
Average_score,4.2,3.85,4.25,3.8,3.7,4.5,3.55,4.05,4.65,3.95,4.0,3.8,3.6,3.65,4.35
Level,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [68]:
# Experimental Engagement: 
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[21,25,26,30,31,32,33,34,36,38]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,10,38,47,39,39,39,34,37,45,42,50,39,44,37,41
Average_score,1.0,3.8,4.7,3.9,3.9,3.9,3.4,3.7,4.5,4.2,5.0,3.9,4.4,3.7,4.1
Level,Low,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [69]:
# Experimental Ability: 
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[22,23,24,27,28,29,35,37,39,40]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,10,38,46,38,34,38,31,42,42,40,46,41,33,34,45
Average_score,1.0,3.8,4.6,3.8,3.4,3.8,3.1,4.2,4.2,4.0,4.6,4.1,3.3,3.4,4.5
Level,Low,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [70]:
# Experimental Overall
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=list(range(21,41))
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,20,76,93,77,73,77,65,79,87,82,96,80,77,71,86
Average_score,1.0,3.8,4.65,3.85,3.65,3.85,3.25,3.95,4.35,4.1,4.8,4.0,3.85,3.55,4.3
Level,Low,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [ ]:
Total_score	20	76	93	77	73	77	65	79	87	82	96	80	77	71	86
Average_score	1.0	3.8	4.65	3.85	3.65	3.85	3.25	3.95	4.35	4.1	4.8	4.0	3.85	3.55	4.3
Level	Low	High	High	High	High	High	High	High	High	High	High	High	High	High	High

## Cognitive Reflection Test

In [62]:
scale_name='Cognitive Reflection Test'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,5,5,10,10,5,10,10,1,10,40,5,50,10,10,100
2,5,5,100,100,5,5,5,1000,50,250,25,250,5,5,5
3,47,47,24,24,47,24,47,24,47,23,96,24,24,24,48
4,4,4,9,9,6,4,4,12,6,4,20,13,9,6,12
5,29,29,30,30,30,30,29,30,15,50,20,150,30,25,15
6,20,20,20,20,10,10,20,10,30,100,10,20,10,30,90
7,c,b,b,b,b,b,c,b,b,b,b,b,b,a,b


In [70]:
correct_answers = [5,5,47,4,29,20,'c']
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,7.0,6.00,1.00,1.00,3.00,2.00,6.00,0.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00
Correct_rate,1.0,0.86,0.14,0.14,0.43,0.29,0.86,0.0,0.14,0.14,0.14,0.14,0.14,0.14,0.14


In [71]:
correct_answers = [10,100,24,9,30,10,'b']
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,0.0,1.00,6.00,6.00,3.00,5.00,1.00,4.00,2.00,1.00,2.00,2.00,6.00,2.00,1.00
Correct_rate,0.0,0.14,0.86,0.86,0.43,0.71,0.14,0.57,0.29,0.14,0.29,0.29,0.86,0.29,0.14


In [ ]:
Correct_number	0.0	1.00	6.00	6.00	3.00	5.00	1.00	4.00	2.00	1.00	2.00	2.00	6.00	2.00	1.00
Correct_rate	0.0	0.14	0.86	0.86	0.43	0.71	0.14	0.57	0.29	0.14	0.29	0.29	0.86	0.29	0.14

## Letter Sets Test

In [71]:
scale_name='Letter Sets Test'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,a,c,a,d,d,a,e,a,e,e,e,e,c,e,e
2,d,d,e,d,e,a,d,d,e,e,e,e,d,d,a
3,d,d,d,d,d,d,d,d,d,e,d,e,d,e,e
4,c,c,e,d,c,a,c,d,e,e,e,e,c,e,a
5,e,c,e,e,d,e,c,d,d,e,a,e,c,e,d
6,c,c,d,d,e,e,c,d,d,e,e,a,e,d,a
7,a,e,e,d,e,a,a,a,e,e,e,a,c,e,e
8,d,c,d,d,e,a,b,a,e,e,e,a,b,e,d
9,d,c,e,d,e,a,b,b,e,e,e,a,c,e,c


In [72]:
correct_answers = ['a','d','d','c','b','c','b','d','e','d','a','d','a','c','c','e','b','d','c','d','b','b','e','e','e','b','a','e','b','a']
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
# dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_rate,0.7,0.4,0.37,0.3,0.3,0.23,0.33,0.33,0.27,0.2,0.23,0.1,0.37,0.3,0.13


In [ ]:
Correct_rate	0.7	0.4	0.37	0.3	0.3	0.23	0.33	0.33	0.27	0.2	0.23	0.1	0.37	0.3	0.13

## Logiqa 2.0

In [70]:
# expname='gpt4'
# expname='gpt35'
# expname='davinci003'
# expname='davinci002'
# expname='claudeinstant'
# expname='bison001'
# expname='bard'

# expname='llama13b'
# expname='vicuna13b'
# expname='openchat35'
expname='wizardlm13b'
# expname='vicuna7b'
# expname='oasst12b'
# expname='dolly12b'
# expname='llama7b'
# expname='chatglm26b'

In [71]:
# load ground truth
with open(os.path.join(filepath,'LogiQA_selected_100.json'),'r') as f:
    data=json.load(f)
truth=['abcd'[x['answer']] for x in data]
len(truth)

100

In [72]:
cates=[]
for x in data:
    cates.extend(list(x['type'].keys()))
    
set(cates)

{'Categorical Reasoning',
 'Conjunctive Reasoning',
 'Disjunctive Reasoning',
 'Necessry Condtional Reasoning',
 'Sufficient Conditional Reasoning'}

In [61]:

with open(os.path.join(filepath,f'answers_{expname}_LogiQA.json'),'r') as f:
    answers=json.load(f)
len(answers)

100

In [62]:
for cate in ['Categorical Reasoning','Sufficient Conditional Reasoning','Necessry Condtional Reasoning','Disjunctive Reasoning', 'Conjunctive Reasoning']:
    idxs=[i for i,task in enumerate(data) if cate in task['type']]
    tmptruth=[truth[i] for i in idxs]
    tmpans=[answers[i] for i in idxs]
    assert len(tmptruth)==len(tmpans)
    cnt=0
    for i in range(len(tmptruth)):
        if tmptruth[i] == tmpans[i]:
            cnt += 1
    rate=cnt/len(tmptruth)
    print('%.2f'%rate)
# overall
tmptruth=truth
tmpans=answers
cnt=0
for i in range(len(tmptruth)):
    if tmptruth[i] == tmpans[i]:
        cnt += 1
rate=cnt/len(tmptruth)
print('%.2f'%rate)

0.19
0.23
0.24
0.25
0.21
0.23


In [ ]:
0.19
0.23
0.24
0.25
0.21
0.23

##  Causal Reasoning Test on LLM

In [73]:
# load ground truth
with open(os.path.join(filepath,'cladder_selected_100.json'),'r') as f:
    data=json.load(f)
truth=[x['ground_truth'] for x in data]
len(truth)

100

In [74]:
# expname='gpt4'
# expname='gpt35'
with open(os.path.join(filepath,f'answers_{expname}_cladder.json'),'r') as f:
    answers=json.load(f)
len(answers)

100

In [75]:
# overall
cnt=0
for i in range(len(tmptruth)):
    if truth[i] == answers[i]:
        cnt += 1
rate=cnt/len(truth)
print('%.2f'%rate)

0.42


## Scientific Reasoning Scale


In [73]:
scale_name='Scientific Reasoning Scale'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,1
2,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1
3,1,1,0,0,0,1,0,1,0,1,0,1,0,1,1
4,0,1,0,0,0,0,0,1,0,1,0,1,0,1,1
5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1
7,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0
8,0,1,0,0,0,0,0,1,1,1,0,1,0,1,1
9,0,0,0,0,1,1,0,1,0,1,0,1,0,1,1


In [74]:
correct_answers = [1,0,1,0,0,0,1,0,0,0,0]
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
# dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_rate,0.91,0.45,0.73,0.73,0.64,0.82,0.82,0.45,0.64,0.27,0.73,0.55,0.64,0.36,0.27


In [ ]:
Correct_rate	0.91	0.45	0.73	0.73	0.64	0.82	0.82	0.45	0.64	0.27	0.73	0.55	0.64	0.36	0.27

## Defeasible Reasoning

In [75]:
scale_name='Defeasible Reasoning'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,b,c,a,b,a,a,a,a,b,a,b,a,b,b,c
2,a,c,a,a,b,a,b,a,a,a,a,a,a,b,c
3,c,b,a,b,b,a,c,a,a,a,a,a,b,b,c
4,a,a,a,b,a,b,a,a,a,a,b,a,a,b,c
5,a,a,a,a,a,a,b,a,a,a,b,a,b,b,c
6,a,a,b,b,a,b,c,a,a,a,b,a,b,b,c


In [76]:
# overall
def check_answers(student_answers):
    correct_count = 0
    for i, answer in enumerate(student_answers):
        # 对于有多个正确答案的题目，检查学生答案是否在其中
        if isinstance(correct_answers[i], list):
            if answer in correct_answers[i]:
                correct_count += 1
        # 对于只有一个正确答案的题目，直接比较
        else:
            if answer == correct_answers[i]:
                correct_count += 1
    return correct_count

correct_answers = ['b','c',['b','c'],'c','c','c']
# correct_answers = ['b','c','c','c','c','c']
correct_count = dfscale.apply(check_answers, axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,2.00,2.00,0.0,2.00,1.00,0.0,2.00,0.0,1.00,0.0,1.00,0.0,2.00,2.00,5.00
Correct_rate,0.33,0.33,0.0,0.33,0.17,0.0,0.33,0.0,0.17,0.0,0.17,0.0,0.33,0.33,0.83


In [77]:
# strong correlation
correct_answers = ['b','c','c','c','c','c']
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,2.00,1.00,0.0,1.00,0.0,0.0,2.00,0.0,1.00,0.0,1.00,0.0,1.00,1.00,5.00
Correct_rate,0.33,0.17,0.0,0.17,0.0,0.0,0.33,0.0,0.17,0.0,0.17,0.0,0.17,0.17,0.83


In [78]:
# weak correlation
correct_answers = ['b','c','b','c','c','c']
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,1.00,2.00,0.0,2.00,1.00,0.0,1.00,0.0,1.00,0.0,1.00,0.0,2.00,2.00,4.00
Correct_rate,0.17,0.33,0.0,0.33,0.17,0.0,0.17,0.0,0.17,0.0,0.17,0.0,0.33,0.33,0.67


In [ ]:
Correct_number	1.00	2.00	0.0	2.00	1.00	0.0	1.00	0.0	1.00	0.0	1.00	0.0	2.00	2.00	4.00
Correct_rate	0.17	0.33	0.0	0.33	0.17	0.0	0.17	0.0	0.17	0.0	0.17	0.0	0.33	0.33	0.67

## V2 Defeasible Reasoning

In [92]:
scale_name='V2 Defeasible Reasoning'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,c,b,b,c,b,b,b,b,c,a,c,a,c,c,b
2,c,a,a,c,a,c,a,a,c,a,a,a,c,c,b
3,a,b,a,c,b,b,b,a,b,a,a,a,b,b,b
4,b,b,b,c,b,c,b,a,c,a,b,a,c,c,b
5,a,a,a,b,a,b,c,a,b,a,b,a,b,b,b
6,c,c,c,c,a,b,a,a,c,a,c,a,c,c,b


In [93]:
# overall
def check_answers(student_answers):
    correct_count = 0
    for i, answer in enumerate(student_answers):
        # 对于有多个正确答案的题目，检查学生答案是否在其中
        if isinstance(correct_answers[i], list):
            if answer in correct_answers[i]:
                correct_count += 1
        # 对于只有一个正确答案的题目，直接比较
        else:
            if answer == correct_answers[i]:
                correct_count += 1
    return correct_count

correct_answers = ['c','b',['b','c'],'a','c','b']
# correct_answers = ['b','c','c','c','c','c']
correct_count = dfscale.apply(check_answers, axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,1.00,1.00,0.0,2.00,1.00,2.00,2.00,1.00,2.00,1.00,1.00,1.00,2.00,2.00,3.0
Correct_rate,0.17,0.17,0.0,0.33,0.17,0.33,0.33,0.17,0.33,0.17,0.17,0.17,0.33,0.33,0.5


In [94]:
# strong correlation
correct_answers = ['c','b','c','a','c','b']
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,1.00,0.0,0.0,2.00,0.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,2.00
Correct_rate,0.17,0.0,0.0,0.33,0.0,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.33


In [95]:
# weak correlation
correct_answers = ['c','b','b','a','c','b']
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,1.00,1.00,0.0,1.00,1.00,2.00,2.00,1.00,2.00,1.00,1.00,1.00,2.00,2.00,3.0
Correct_rate,0.17,0.17,0.0,0.17,0.17,0.33,0.33,0.17,0.33,0.17,0.17,0.17,0.33,0.33,0.5


In [ ]:
Correct_number	1.00	1.00	0.0	1.00	1.00	2.00	2.00	1.00	2.00	1.00	1.00	1.00	2.00	2.00	3.0
Correct_rate	0.17	0.17	0.0	0.17	0.17	0.33	0.33	0.17	0.33	0.17	0.17	0.17	0.33	0.33	0.5

## Wason Selection Task 1

In [79]:
scale_name='Wason Selection Task'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,ac,ad,ac,ac,a,a,abd,ab,ac,a,ac,ab,ac,a,a
2,ad,ad,ad,acd,ad,ad,a,ab,ab,acd,ad,ac,a,d,ab


In [84]:
ids=[1]
correct_answers = ['ad']
correct_count = dfscale.astype(str).loc[ids,:].apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)

dfresult.loc['Correct_number'] = correct_count.astype(int)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [85]:
ids=[2]
correct_answers = ['ad']
correct_count = dfscale.astype(str).loc[ids,:].apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)

dfresult.loc['Correct_number'] = correct_count.astype(int)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0


## V2 Wason Selection Task

In [96]:
scale_name='V2 Wason Selection Task'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,ac,ac,ac,ac,a,a,a,abcd,ac,ac,ab,ab,ac,ac,a
2,ad,ad,ad,acd,ac,a,ad,ad,ad,a,ad,ac,ad,ad,abcd
3,ad,ad,ad,d,ab,ad,ad,ac,ad,ad,ad,ad,d,a,d
4,ab,ab,bc,bc,bc,bd,ab,ab,bc,bc,bc,bc,b,a,ab
5,ad,ad,ad,d,ab,ad,ad,a,ad,ad,ad,ad,d,a,d
6,bc,bc,bc,cd,bc,cd,bc,bc,bc,bc,bc,bc,c,a,ab


In [97]:
# odd
ids=[1,3,5]
correct_answers = ['ad','ab','ab']
correct_count = dfscale.astype(str).loc[ids,:].apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)

dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,0.0,0.0,0.0,0.0,2.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Correct_rate,0.0,0.0,0.0,0.0,0.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
# even
ids=[2,4,6]
correct_answers = ['ad','ab','cd']
correct_count = dfscale.astype(str).loc[ids,:].apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)

dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,2.00,2.00,1.00,1.00,0.0,1.00,2.00,2.00,1.00,0.0,1.00,0.0,1.00,1.00,1.00
Correct_rate,0.67,0.67,0.33,0.33,0.0,0.33,0.67,0.67,0.33,0.0,0.33,0.0,0.33,0.33,0.33


In [101]:
# overall
ids=dfscale.index
correct_answers = ['ad','ad','ab','ab','ab','cd']
correct_count = dfscale.astype(str).loc[ids,:].apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)

dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,2.00,2.00,1.00,1.00,2.00,1.00,2.00,2.00,1.00,0.0,1.00,0.0,1.00,1.00,1.00
Correct_rate,0.33,0.33,0.17,0.17,0.33,0.17,0.33,0.33,0.17,0.0,0.17,0.0,0.17,0.17,0.17


In [ ]:
Correct_rate	0.33	0.33	0.17	0.17	0.33	0.17	0.33	0.33	0.17	0.0	0.17	0.0	0.17	0.17	0.17

## Critical Thinking Disposition Scale


In [86]:
scale_name='Critical Thinking Disposition Scale'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale.shape

(11, 15)

In [87]:
# Reflective Skepticism
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=list(range(1,8))
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val <= 3 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,30,31,31,29,29,28,31,33,31,32,28,31,31,30,32
Average_score,4.29,4.43,4.43,4.14,4.14,4.0,4.43,4.71,4.43,4.57,4.0,4.43,4.43,4.29,4.57
Level,High,High,High,High,High,Neutral,High,High,High,High,Neutral,High,High,High,High


In [88]:
# Critical Openness
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=list(range(8,12))
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val <= 3 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,17,19,19,16,17,16,17,19,18,19,17,18,17,18,17
Average_score,4.25,4.75,4.75,4.0,4.25,4.0,4.25,4.75,4.5,4.75,4.25,4.5,4.25,4.5,4.25
Level,High,High,High,Neutral,High,Neutral,High,High,High,High,High,High,High,High,High


In [89]:
# Overall
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val <= 3 else 'High' if val > 4 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,47,50,50,45,46,44,48,52,49,51,45,49,48,48,49
Average_score,4.27,4.55,4.55,4.09,4.18,4.0,4.36,4.73,4.45,4.64,4.09,4.45,4.36,4.36,4.45
Level,High,High,High,High,High,Neutral,High,High,High,High,High,High,High,High,High


In [ ]:
Total_score	47	50	50	45	46	44	48	52	49	51	45	49	48	48	49
Average_score	4.27	4.55	4.55	4.09	4.18	4.0	4.36	4.73	4.45	4.64	4.09	4.45	4.36	4.36	4.45
Level	High	High	High	High	High	Neutral	High	High	High	High	High	High	High	High	High

## Availability Heuristics

In [11]:
scale_name='Availability Heuristics'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,b,b,a,a,b,a,b,a,b,a,b,b,b,b,b
2,b,b,a,a,a,a,b,a,b,a,a,a,a,a,b
3,a,a,a,b,a,a,a,a,a,b,b,b,a,a,b
4,a,a,a,a,a,a,a,a,b,b,a,a,a,b,a


In [13]:
correct_answers = 'bbaa'
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,4.0,4.0,2.0,1.00,3.00,2.0,4.0,2.0,3.00,0.0,2.0,2.0,3.00,2.0,3.00
Correct_rate,1.0,1.0,0.5,0.25,0.75,0.5,1.0,0.5,0.75,0.0,0.5,0.5,0.75,0.5,0.75


## Base-Rate Neglect (Statistical)

In [14]:
scale_name='Base-Rate Neglect (Statistical)'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,a,a,a,a,b,a,a,b,a,a,b,a,b,b,b
2,a,a,a,b,a,a,b,a,a,a,a,b,a,a,b
3,a,b,a,b,a,a,b,a,a,a,a,a,a,a,b
4,a,a,b,b,a,b,b,b,a,a,b,a,b,b,b


In [15]:
correct_answers = 'abbb'
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,1.00,2.0,2.0,4.0,0.0,2.0,4.0,1.00,1.00,1.00,1.00,2.0,1.00,1.00,3.00
Correct_rate,0.25,0.5,0.5,1.0,0.0,0.5,1.0,0.25,0.25,0.25,0.25,0.5,0.25,0.25,0.75


## Base-Rate Neglect (Causal)

In [21]:
scale_name='Base-Rate Neglect (Causal)'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,b,c,c,c,c,c,c,b,b,c,b,c,c,d,b
2,c,c,c,c,c,b,c,c,c,c,b,c,d,b,c
3,b,b,b,c,b,b,b,b,c,c,b,c,c,c,c


In [22]:
id=1
dfscale.loc[id] = dfscale.loc[id].replace({'a': 4, 'b': 3, 'c': 2, 'd': 1})
for id in [2,3]:
    dfscale.loc[id] = dfscale.loc[id].replace({'a': 1, 'b': 2, 'c': 3, 'd': 4})
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,3,2,2,2,2,2,2,3,3,2,3,2,2,1,3
2,3,3,3,3,3,2,3,3,3,3,2,3,4,2,3
3,2,2,2,3,2,2,2,2,3,3,2,3,3,3,3


In [24]:

dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
# dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)


dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Average_score,2.67,2.33,2.33,2.67,2.33,2.0,2.33,2.67,3.0,2.67,2.33,2.67,3.0,2.0,3.0


In [ ]:
Average_score	2.67	2.33	2.33	2.67	2.33	2.0	2.33	2.67	3.0	2.67	2.33	2.67	3.0	2.0	3.0

## Conjunction Fallacy

In [102]:
scale_name='Conjunction Fallacy'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,a,b,b,b,b,a,b,b,b,b,b,b,b,b,b
2,a,b,b,b,b,b,b,b,b,b,b,b,b,a,b
3,b,b,b,b,a,b,b,a,b,a,b,b,b,a,a
4,b,a,a,a,a,a,b,a,a,a,a,a,b,a,a
5,a,b,b,b,b,b,b,b,a,b,b,b,b,b,b
6,b,a,a,b,a,a,a,a,a,a,a,a,b,a,a


In [104]:
correct_answers = ['a','a','b','b','a','b']
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,6.0,1.00,1.00,2.00,0.0,2.00,2.00,0.0,2.00,0.0,1.00,1.00,3.0,1.00,0.0
Correct_rate,1.0,0.17,0.17,0.33,0.0,0.33,0.33,0.0,0.33,0.0,0.17,0.17,0.5,0.17,0.0


In [ ]:
Correct_number	6.0	1.00	1.00	2.00	0.0	2.00	2.00	0.0	2.00	0.0	1.00	1.00	3.0	1.00	0.0
Correct_rate	1.0	0.17	0.17	0.33	0.0	0.33	0.33	0.0	0.33	0.0	0.17	0.17	0.5	0.17	0.0

## Framing Effect

### Framing Effect: Risk Framing

In [56]:
scale_name='Framing Effect: Risk Framing'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,3,1,5,6,5,4,4,5,5,1,4,5,1,1,5
2,3,1,4,4,4,1,3,4,4,4,3,4,1,6,4
3,2,6,4,4,2,1,4,4,4,4,3,5,3,1,5
4,3,1,3,6,3,1,2,4,5,4,3,6,3,6,5
5,3,2,5,1,3,5,4,4,4,6,3,6,1,4,5
6,3,2,6,6,3,1,2,3,5,4,3,1,1,6,6
7,3,6,5,6,1,1,4,5,5,3,3,5,6,1,5
8,3,6,5,6,4,6,4,3,4,6,3,6,1,6,5
9,3,1,3,3,2,1,2,3,1,3,3,3,2,6,5


In [57]:
pairs=[(1,5),(2,4),(3,7),(4,2),(5,6),(6,3),(7,1)]
dfresult = pd.DataFrame(columns=dfscale.columns)

i,j=pairs[0]
sum_abs_diff = abs(dfscale.loc[i] - dfscale.loc[7+j])
for i,j in pairs[1:]:
    sum_abs_diff += abs(dfscale.loc[i] - dfscale.loc[7+j])
dfresult.loc['Risk']=(5-sum_abs_diff/len(pairs)).astype(float).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Risk,4.71,4.0,3.43,2.86,3.86,4.0,4.71,4.29,3.86,3.0,5.0,2.86,3.43,3.14,4.57


### Framing Effect: Attribute Framing

In [58]:
scale_name='Framing Effect: Attribute Framing'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,5,6,6,6,6,6,5,1,6,6,6,6,6,6,5
2,4,4,5,4,4,5,4,5,5,5,5,5,3,5,5
3,4,3,3,3,4,4,4,4,4,4,4,4,3,3,5
4,3,3,3,3,4,3,3,5,4,4,4,4,3,4,4
5,4,4,6,4,5,5,5,5,4,4,4,4,4,5,5
6,2,2,3,3,3,3,3,4,4,4,3,4,3,3,4
7,5,6,6,6,4,5,4,5,6,5,6,4,3,6,4
8,3,3,3,3,3,3,3,5,4,4,4,2,3,3,4
9,4,4,5,3,4,4,4,3,4,5,4,4,4,4,4


In [59]:
pairs=[(1,6),(2,5),(3,3),(4,1),(5,7),(6,2),(7,4)]
# dfresult = pd.DataFrame(columns=dfscale.columns)

i,j=pairs[0]
sum_abs_diff = abs(dfscale.loc[i] - dfscale.loc[7+j])
for i,j in pairs[1:]:
    sum_abs_diff += abs(dfscale.loc[i] - dfscale.loc[7+j])
dfresult.loc['Attribute']=(5-sum_abs_diff/len(pairs)).astype(float).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Risk,4.71,4.00,3.43,2.86,3.86,4.00,4.71,4.29,3.86,3.00,5.00,2.86,3.43,3.14,4.57
Attribute,4.00,3.29,3.57,4.00,4.00,3.86,4.29,4.29,3.86,4.71,4.29,3.57,4.00,3.71,4.57


In [62]:
dfresult.loc['Overall']=dfresult.mean().round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Risk,4.71,4.00,3.43,2.86,3.86,4.00,4.71,4.29,3.86,3.00,5.00,2.86,3.43,3.14,4.57
Attribute,4.00,3.29,3.57,4.00,4.00,3.86,4.29,4.29,3.86,4.71,4.29,3.57,4.00,3.71,4.57
Overall,4.36,3.64,3.50,3.43,3.93,3.93,4.50,4.29,3.86,3.86,4.64,3.22,3.72,3.42,4.57


In [ ]:
4.36	3.64	3.50	3.43	3.93	3.93	4.50	4.29	3.86	3.86	4.64	3.22	3.72	3.42	4.57

## Outcome bias

In [52]:
scale_name='Outcome bias'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,1,1,3,3,4,1,2,2,4,4,5,4,3,6,5
2,4,4,4,3,4,4,4,4,4,4,5,5,5,6,4
3,6,6,6,6,5,5,5,4,4,5,6,5,6,6,4
4,4,3,2,4,4,3,3,5,4,4,3,5,3,4,4
5,4,6,6,6,5,5,5,6,6,5,6,5,6,6,5
6,2,3,2,1,3,1,3,5,2,4,3,3,1,1,4
7,6,6,6,6,4,6,5,4,6,5,6,6,6,6,6
8,4,3,4,1,4,3,2,4,2,3,4,6,3,1,5


In [55]:
dfresult = pd.DataFrame(columns=dfscale.columns)

sum_abs_diff = dfscale.loc[2] - dfscale.loc[1]
sum_abs_diff += dfscale.loc[3] - dfscale.loc[4]
sum_abs_diff += dfscale.loc[5] - dfscale.loc[6]
sum_abs_diff += dfscale.loc[7] - dfscale.loc[8]

dfresult.loc['Average_score']=(5-sum_abs_diff/4).astype(float).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Average_score,2.75,2.0,2.25,2.0,4.25,2.0,2.75,4.5,3.0,4.0,3.0,4.25,1.75,2.0,4.75


In [ ]:
Average_score	2.75	2.0	2.25	2.0	4.25	2.0	2.75	4.5	3.0	4.0	3.0	4.25	1.75	2.0	4.75

## Probabilistic Matching

In [105]:
scale_name='Probabilistic Matching'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,d,c,e,d,c,c,c,c,e,b,c,b,b,c,e
2(number of b),3,3,4,3,3,3,4,5,5,5,0,5,0,4,4


In [106]:
correct_answers = ['d','0']
correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: sum(x == correct_answers), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Correct_number'] = correct_count.astype(int)
dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Correct_number,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
Correct_rate,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0


In [ ]:
Correct_number	1.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	1.0	0.0	0.0

## Sunk Cost Fallacy

In [112]:
scale_name='Sunk Cost Fallacy'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale.shape
# dfscale

(10, 15)

In [110]:
# Rational
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
# cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
# dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,42,43,52,49,41,47,43,22,48,43,49,22,60,48,52
Average_score,4.2,4.3,5.2,4.9,4.1,4.7,4.3,2.2,4.8,4.3,4.9,2.2,6.0,4.8,5.2


In [ ]:
Average_score	4.2	4.3	5.2	4.9	4.1	4.7	4.3	2.2	4.8	4.3	4.9	2.2	6.0	4.8	5.2

## Regret Aversion

In [116]:
scale_name='Regret Aversion'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
# dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,a,a,a,c,a,a,a,a,a,a,a,a,a,a,b
2,c,c,c,c,c,c,c,a,c,a,c,a,c,c,c
3,c,c,c,c,c,c,c,c,c,c,c,c,c,c,b


In [119]:
replacement_dict = {'a': 1, 'b': 1, 'c': 0}
dfscale.replace(replacement_dict, inplace=True)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
# dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,1,1,1,0,1,1,1,2,1,2,1,2,1,1,2


In [ ]:
Total_score	1	1	1	0	1	1	1	2	1	2	1	2	1	1	2

## General Decision-Making Style

In [90]:
scale_name='General Decision-Making Style'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=[3,4,5,7,8,9,12,16,17]
# dfscale.loc[ids] = 6 - dfscale.loc[ids]
dfscale.fillna(0, inplace=True)
dfscale.shape

(25, 15)

In [91]:
# Rational
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[4,7,11,13,25]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,25,20,25,25,20,24,23,25,25,24,23,20,22,24,23
Average_score,5.0,4.0,5.0,5.0,4.0,4.8,4.6,5.0,5.0,4.8,4.6,4.0,4.4,4.8,4.6
Level,High,High,High,High,High,High,High,High,High,High,High,High,High,High,High


In [92]:
# Avoidant
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[6,14,19,21,23]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,10,10,5,13,10,9,11,10,8,15,7,16,25,17,17
Average_score,2.0,2.0,1.0,2.6,2.0,1.8,2.2,2.0,1.6,3.0,1.4,3.2,5.0,3.4,3.4
Level,Low,Low,Low,Low,Low,Low,Low,Low,Low,Neutral,Low,High,High,High,High


In [93]:
# Dependent
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[2,5,10,18,22]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,13,16,18,12,18,18,14,17,14,22,19,20,22,19,22
Average_score,2.6,3.2,3.6,2.4,3.6,3.6,2.8,3.4,2.8,4.4,3.8,4.0,4.4,3.8,4.4
Level,Low,High,High,Low,High,High,Low,High,Low,High,High,High,High,High,High


In [94]:
# Intuitive
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[1,3,12,16,17]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,18,23,21,11,14,21,20,24,20,21,21,24,19,22,21
Average_score,3.6,4.6,4.2,2.2,2.8,4.2,4.0,4.8,4.0,4.2,4.2,4.8,3.8,4.4,4.2
Level,High,High,High,Low,Low,High,High,High,High,High,High,High,High,High,High


In [95]:
# Spontaneous
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=[8,9,15,20,24]
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,11,10,24,6,10,17,15,14,11,18,14,19,24,20,19
Average_score,2.2,2.0,4.8,1.2,2.0,3.4,3.0,2.8,2.2,3.6,2.8,3.8,4.8,4.0,3.8
Level,Low,Low,High,Low,Low,High,Neutral,Low,Low,High,Low,High,High,High,High


In [96]:
# Overall
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

mean_value = dfresult.loc['Average_score']
cate_row = ['Low' if val < 3 else 'High' if val > 3 else 'Neutral' for val in mean_value]
dfresult.loc['Level'] = cate_row

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,77,79,93,67,72,89,83,90,78,100,84,99,112,102,102
Average_score,3.08,3.16,3.72,2.68,2.88,3.56,3.32,3.6,3.12,4.0,3.36,3.96,4.48,4.08,4.08
Level,High,High,High,Low,Low,High,High,High,High,High,High,High,High,High,High


In [ ]:
Total_score	77	79	93	67	72	89	83	90	78	100	84	99	112	102	102
Average_score	3.08	3.16	3.72	2.68	2.88	3.56	3.32	3.6	3.12	4.0	3.36	3.96	4.48	4.08	4.08
Level	High	High	High	Low	Low	High	High	High	High	High	High	High	High	High	High

## Actively Open-Minded Thinking Scale

In [10]:
scale_name='Actively Open-Minded Thinking Scale'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
ids=list(range(1,16))
dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
# dfscale

(15, 15)

In [11]:
# Overall
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
# dfresult.loc['Total_score'] = dfscale.loc[ids].sum()
dfresult.loc['Average_score'] = dfscale.loc[ids].astype(float).mean().round(2)

dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Average_score,4.27,4.53,4.6,4.27,3.8,2.8,4.2,3.2,3.73,4.2,4.2,5.13,1.93,4.67,1.4


In [ ]:
Average_score	4.27	4.53	4.6	4.27	3.8	2.8	4.2	3.2	3.73	4.2	4.2	5.13	1.93	4.67	1.4

## Belief Bias in Syllogistic Reasoning

In [32]:
scale_name='Belief Bias in Syllogistic Reasoning'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,a,a,a,a,a,a,a,a,a,a,a,a,a,a,b
2,b,b,a,a,b,a,a,a,b,b,b,a,a,a,b
3,b,b,b,b,b,b,b,b,b,b,b,a,b,a,b
4,b,a,a,a,b,a,b,b,b,b,b,a,a,a,b
5,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b
6,a,b,a,a,b,a,b,b,b,b,b,b,a,a,b
7,a,a,a,a,a,a,a,b,a,b,a,a,a,a,b
8,b,b,b,b,b,b,b,b,b,b,b,a,b,a,b


In [33]:
def cal_score(x):
    score = 0
    if x[1]=='a' and x[2]=='b':
        score += 1
    if x[3]=='b' and x[4]=='a':
        score += 1
    if x[5]=='b' and x[6]=='a':
        score += 1
    if x[7]=='a' and x[8]=='b':
        score += 1
    return score

# correct_answers = ['d','0']
# correct_answers = [str(x) for x in correct_answers]
correct_count = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = correct_count.astype(int)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,3,3,3,3,2,3,1,0,2,1,2,0,3,1,0


In [34]:
correct_count

gpt-4               3
gpt-3.5             3
text-davinci-003    3
text-davinci-002    3
claude-instant      2
text-bison-001      3
bard                1
chatglm2-6b         0
llama2-13b          2
llama2-7b           1
vicuna-13b          2
vicuna-7b           0
openchat-13b        3
wizardlm-13b        1
oasst-12b           0
dtype: int64

## Bias Blind Spot

In [36]:
scale_name='Bias Blind Spot'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
# dfscale

(28, 15)

In [39]:
def cal_score(x):
    score = []
    for i in range(1,15):
        tmp = x[i+14] - x[i]
        score.append(tmp)
    score = np.mean(score)
    
    return score


score = dfscale.astype(float).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,2.0,1.29,3.29,1.29,0.79,1.86,0.21,0.71,1.14,2.36,0.29,0.64,1.79,0.71,-0.29


In [ ]:
Total_score	2.0	1.29	3.29	1.29	0.79	1.86	0.21	0.71	1.14	2.36	0.29	0.64	1.79	0.71	-0.29

## Hindsight Bias

In [42]:
scale_name='Hindsight Bias'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,d,c,c,c,c,d,d,c,d,c,d,c,c,d,b


In [43]:
def cal_score(x):
    score_map = {'a':4, 'b':3, 'c':2, 'd':1}
    score = score_map[x[1]]
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,1,2,2,2,2,1,1,2,1,2,1,2,2,1,3


In [ ]:
Total_score	1	2	2	2	2	1	1	2	1	2	1	2	2	1	3

## Illusion of control

In [44]:
scale_name='Illusion of control'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,b,a,a,a,b,a,b,a,a,a,a,a,a,a,a
2,b,a,b,b,b,a,b,a,b,a,b,a,b,a,a


In [48]:
def cal_score(x):
    score = (x.tolist().count('a'))
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,0,2,1,1,0,2,0,2,1,2,1,2,1,2,2


In [ ]:
Total_score	0	2	1	1	0	2	0	2	1	2	1	2	1	2	2

## Better-than-Average Effect

In [49]:
scale_name='Better-than-Average Effect'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,b,c,c,c,b,b,c,c,b,d,b,b,b,b,d
2,c,c,c,d,c,c,c,c,c,d,c,c,c,c,b
3,c,c,b,c,b,b,b,c,b,c,c,c,c,c,b


In [50]:
def cal_score(x):
    score_map = {'a':1, 'b':2, 'c':3, 'd':4}
    score = 0
    score += score_map[x[1]] - 2
    score += score_map[x[2]] - 3
    score += score_map[x[3]] - 2
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,1,2,1,3,0,0,1,2,0,4,1,1,1,1,1


In [ ]:
Total_score	1	2	1	3	0	0	1	2	0	4	1	1	1	1	1

## Confirmation Bias


In [51]:
scale_name='Confirmation Bias'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,b,a,b,b,b,a,b,b,a,a,b,a,b,a,b
2,b,b,a,b,b,b,b,b,b,a,b,b,b,b,b
3,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b
4,b,b,b,b,b,b,b,b,b,a,b,b,b,b,b
5,b,b,b,b,b,b,b,b,b,a,b,b,b,b,b


In [52]:
def cal_score(x):
    score_map = {'a':1, 'b':0, 'c':-1}
    score = sum([score_map[t] for t in x.values])
    score += 2
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,2,3,3,2,2,3,2,2,3,6,2,3,2,3,2


In [ ]:
Total_score	2	3	3	2	2	3	2	2	3	6	2	3	2	3	2

## Covariation detection


In [54]:
scale_name='Covariation detection'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,-2,8,7,10,7,8,7,10,8,8,8,10,1,6,10


In [56]:
def cal_score(x):
    if x[1]<0:
        score = 1
    else:
        score = 0
    return score

score = dfscale.astype(int).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
1	0	0	0	0	0	0	0	0	0	0	0	0	0	0

## Denominator neglect

In [57]:
scale_name='Denominator neglect'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,small,large,large,large,large,small,small,small,large,small,large,large,large,small,large


In [58]:
def cal_score(x):
    if x[1] == 'small':
        score = 1
    else:
        score = 0
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,1,0,0,0,0,1,1,1,0,1,0,0,0,1,0


In [ ]:
1	0	0	0	0	1	1	1	0	1	0	0	0	1	0

## Gambler’s Fallacy

In [60]:
scale_name='Gambler’s Fallacy'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
# dfscale

(10, 15)

In [61]:
def cal_score(x):
    correct_answer = ['c','c','a','c','a','a','a','d','a','a']
    score = 0
    for i,j in zip(x.values,correct_answer):
        if i==j:
            score+=1
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,9,4,6,5,6,6,6,2,2,0,5,4,3,5,3


In [ ]:
Total_score	9	4	6	5	6	6	6	2	2	0	5	4	3	5	3

## Loss Aversion

In [62]:
scale_name='Loss Aversion'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,b,a,a,a,a,a,a,a,a,b,a,b,a,a,a
2,a,a,a,a,a,b,b,a,a,b,a,b,a,a,a


In [63]:
def cal_score(x):
    score = x.tolist().count('b')
    score /= 2
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Average_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Average_score,0.5,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
Average_score	0.5	0.0	0.0	0.0	0.0	0.5	0.5	0.0	0.0	1.0	0.0	1.0	0.0	0.0	0.0

## Mental Accounting

In [64]:
scale_name='Mental Accounting'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,a,a,a,a,a,a,a,b,a,a,a,a,a,a,a
2,b,b,a,a,b,b,b,b,b,a,a,a,a,a,b


In [65]:
def cal_score(x):
    if x[1]=='a' and x[2]=='b':
        score = 1
    else:
        score = 0
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,1,1,0,0,1,1,1,0,1,0,0,0,0,0,1


In [ ]:
1	1	0	0	1	1	1	0	1	0	0	0	0	0	1

## Regression to the mean

In [66]:
scale_name='Regression to the mean'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,c,c,a,c,a,c,c,c,c,e,a,c,c,c,e


In [67]:
def cal_score(x):
    score = int(x[1]=='c')
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,1,1,0,1,0,1,1,1,1,0,0,1,1,1,0


In [ ]:
1	1	0	1	0	1	1	1	1	0	0	1	1	1	0

## Temporal Discounting

In [68]:
scale_name='Temporal Discounting'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,f,b,c,c,b,d,c,f,c,b,d,c,c,c,b
2,d,c,c,c,d,a,c,f,c,b,c,b,c,b,f
3,d,c,c,d,d,d,c,f,c,e,c,d,f,d,c
4,f,c,c,d,d,d,d,f,c,b,d,d,c,d,f
5,d,c,c,d,c,d,c,f,c,a,c,d,c,d,f


In [72]:
def cal_score(x):
    tmp='abcdef'
    score_map = {t:i+1 for i,t in enumerate(tmp)}
#     print(score_map)
    score = sum([score_map[t] for t in x.values])
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Total_score'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Total_score,24,14,15,18,17,17,16,30,15,12,17,17,18,17,23


In [ ]:
24	14	15	18	17	17	16	30	15	12	17	17	18	17	23

## Overconfidence

In [74]:
scale_name='Overconfidence'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
# dfscale

(36, 15)

In [80]:
answer = {
    "1": "B",
    "3": "B",
    "5": "A",
    "7": "A",
    "9": "A",
    "11": "A",
    "13": "B",
    "15": "A",
    "17": "A",
    "19": "B",
    "21": "B",
    "23": "A",
    "25": "B",
    "27": "B",
    "29": "B",
    "31": "A",
    "33": "A",
    "35": "B"
}
def cal_score(x):
    correct = 0
    for i in range(1,37,2):
        if x[i] == answer[str(i)].lower():
            correct += 1
    correct /= 18
    
    confidence = []
    for i in range(2,37,2):
        confidence.append(int(x[i])/100)
    confidence = np.mean(confidence)
    score = np.abs(confidence - correct)
    return score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Average_bias'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Average_bias,0.08,0.1,0.33,0.14,0.07,0.04,0.07,0.33,0.17,0.21,0.17,0.43,0.08,0.08,0.43


In [ ]:
Average_biasa	0.08	0.1	0.33	0.14	0.07	0.04	0.07	0.33	0.17	0.21	0.17	0.43	0.08	0.08	0.43

## v2 Endowment Effect 

In [81]:
scale_name='v2 Endowment Effect '
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,b,b,a,a,b,a,b,b,b,a,a,b,b,b,b
2,b,b,a,a,b,a,b,a,b,b,a,b,a,a,b
3,b,b,a,a,b,a,b,a,b,a,a,b,a,a,b
4,b,b,a,a,b,a,b,a,b,b,a,b,a,a,b
5,b,b,a,a,b,a,b,a,b,a,a,b,a,a,b
6,b,b,a,a,b,a,b,a,b,b,a,b,a,a,b
7,b,b,a,a,b,a,b,a,b,a,a,b,a,a,b
8,b,a,a,a,b,a,b,a,b,b,a,b,a,a,b
9,b,a,a,a,b,a,b,a,b,a,a,b,a,a,b


In [92]:
def is_b_before_a(x):
    a_seen = False  # 用于跟踪是否已经遇到了元素a
    for element in x:
        if element == 'a':
            a_seen = True
        elif element == 'b' and a_seen:
            return False  # 如果在遇到a之前出现了b，则返回False
    return True  # 如果没有在遇到a之前出现b，则返回True

def is_a_before_b(x):
    b_seen = False  # 用于跟踪是否已经遇到了元素a
    for element in x:
        if element == 'b':
            b_seen = True
        elif element == 'a' and b_seen:
            return False  # 如果在遇到a之前出现了b，则返回False
    return True  # 如果没有在遇到a之前出现b，则返回True

def cal_score(x):
    wtalist = x.tolist()[:20]
    wtplist = x.tolist()[20:]
    error = False
#     print(wtalist,wtplist)
    if is_b_before_a(wtalist):
        if 'a' in wtalist:
            pos = wtalist.index('a')
            if pos == 0:
                wta = 0
            else:
                wta = pos/2-0.25
        else:
            wta = 9.5
    else:
        error = True
        
    if is_a_before_b(wtplist):
        if 'b' in wtplist:
            pos = wtplist.index('b')
            if pos == 0:
                wtp = 0
            else:
                wtp = pos/2-0.25
        else:
            wtp = 9.5
    else:
        error = True
        
    if error:
        wta = -1
        wtp = -1
        score = 9.5
    else:
        score = np.abs(wta-wtp)
        
    return wta, wtp, score

score = dfscale.astype(str).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['WTA'] = dfscale.astype(str).apply(lambda x: cal_score(x)[0], axis=0).round(2)
dfresult.loc['WTP'] = dfscale.astype(str).apply(lambda x: cal_score(x)[1], axis=0).round(2)
dfresult.loc['|WTA-WTP|'] = dfscale.astype(str).apply(lambda x: cal_score(x)[2], axis=0).round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
WTA,6.25,3.25,-1.0,-1.0,6.25,-1.0,8.75,-1.0,-1.0,-1.0,-1.0,9.5,0.25,0.25,9.5
WTP,6.25,5.75,-1.0,-1.0,9.50,-1.0,2.25,-1.0,-1.0,-1.0,-1.0,0.0,9.50,9.50,0.0
|WTA-WTP|,0.00,2.50,9.5,9.5,3.25,9.5,6.50,9.5,9.5,9.5,9.5,9.5,9.25,9.25,9.5


In [ ]:
WTA	6.25	3.25	-1.0	-1.0	6.25	-1.0	8.75	-1.0	-1.0	-1.0	-1.0	9.5	0.25	0.25	9.5
WTP	6.25	5.75	-1.0	-1.0	9.50	-1.0	2.25	-1.0	-1.0	-1.0	-1.0	0.0	9.50	9.50	0.0
|WTA-WTP|	0.00	2.50	9.5	9.5	3.25	9.5	6.50	9.5	9.5	9.5	9.5	9.5	9.25	9.25	9.5

## v2 Causal Reasoning

In [216]:
scale_name='v2 Causal Reasoning'
dfscale=df[df['scale name']==scale_name]
dfscale=dfscale.drop('scale name',axis=1)
dfscale=dfscale.set_index('id')

# reverse scoring
# ids=list(range(1,16))
# dfscale.loc[ids] = 7 - dfscale.loc[ids]

dfscale.fillna(0, inplace=True)
dfscale.shape
dfscale

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
id,,,,,,,,,,,,,,,
1,0,20,15,10,20,10,10,18,10,10,10,10,18,10,5
2,9,20,10,10,10,1,7.5,11,12,10,40,10,10,0,20
3,16,20,15,14,20,10,17,20,16,10,10,10,16,12,4
4,4,10,8,10,10,0,1,19,10,9,5,10,10,10,10
5,20,20,15,10,20,10,6,6,14,10,10,10,16,20,5
6,0,0,8,10,0,0,0,20,10,10,10,10,10,0,19
7,19,16,15,14,20,10,19,20,16,7,10,10,16,20,10
8,0,0,8,10,0,0,2.5,10,0,2.5,0,10,10,10,0


In [223]:
# common cause
def cal_score(x):
    correct_answer = np.array([10, 10, 16, 4, 16, 4, 16, 4])
#     human_answer = np.array([9.04, 6.26, 14.79, 3.29, 14.04, 3.08, 13.67, 3.08])
#     ans = human_answer
    ans = np.array(x.values)
    score = np.mean(np.abs(ans - correct_answer)/correct_answer)
    return score

score = dfscale.astype(float).apply(lambda x: cal_score(x), axis=0)
dfresult = pd.DataFrame(columns=dfscale.columns)
ids=dfscale.index
dfresult.loc['Average_bias'] = score.round(2)
# dfresult.loc['Correct_rate'] = (correct_count/len(correct_answers)).round(2)
dfresult

,gpt-4,gpt-3.5,text-davinci-003,text-davinci-002,claude-instant,text-bison-001,bard,chatglm2-6b,llama2-13b,llama2-7b,vicuna-13b,vicuna-7b,openchat-13b,wizardlm-13b,oasst-12b
Average_bias,0.44,0.75,0.46,0.64,0.66,0.63,0.41,1.41,0.54,0.55,0.86,0.7,0.66,0.72,1.2


In [ ]:
0.44	0.75	0.46	0.64	0.66	0.63	0.41	1.41	0.54	0.55	0.86	0.7	0.66	0.72	1.2